In [16]:
%load_ext autoreload
%autoreload 2

from notte.env import NotteEnv

from tracing.snapshot_server import SnapshotServer
from tracing.trace_logger import trace_logger, save_snapshot


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# url = "https://www.google.com/travel/flights"
url = "https://example.com"
dir_name = 'test_tracing'

example_steps = [
    lambda page: page.click("text=More information"),
    # lambda page: page.fill("#search-box", "Playwright testing"),
    # lambda page: page.press("#search-box", "Enter"),
]

google_flight_steps = [
    lambda page: page.get_by_role("combobox", name="Where from?").fill("London"),
    lambda page: page.get_by_role("combobox", name="Where to?").fill("Paris"),
    lambda page: page.get_by_role("textbox", name="Departure").fill("2025-01-01"),
    lambda page: page.get_by_role("textbox", name="Return").fill("2025-02-01"),
    lambda page: page.keyboard.press("Enter"),
    lambda page: page.wait_for_timeout(1000),
    lambda page: page.get_by_role("button", name="Search").click(),
    lambda page: page.wait_for_timeout(3000)
]
# Run the async function
await trace_logger(url, example_steps, snapshot_dir=dir_name)

Saved main page as: test_tracing/step0/index.html
Saved resource: test_tracing/step0/index.bin
Saved resource: test_tracing/step0/domains_example.bin
Saved main page as: test_tracing/step1/index.html
Saved resource: test_tracing/step0/_css_2022_iana_website.css
Saved resource: test_tracing/step0/_js_jquery.js
Saved resource: test_tracing/step0/_img_2022_iana-logo-header.svg
Saved resource: test_tracing/step0/_js_iana.js
Saved resource: test_tracing/step1/help_example-domains.bin
Saved resource: test_tracing/step1/_css_2022_iana_website.css
Saved resource: test_tracing/step1/_js_iana.js
Saved resource: test_tracing/step1/_js_jquery.js
Saved resource: test_tracing/step1/_img_2022_iana-logo-header.svg
Saved resource: test_tracing/step1/_img_2022_fonts_NotoSans-Regular.woff
Saved resource: test_tracing/step1/_img_2022_fonts_NotoSans-Bold.woff
Failed to save resource https://www.iana.org/_img/bookmark_icon.ico: Route.fetch: Target page, context or browser has been closed
Call log:
  - → GET

In [15]:

    




from playwright.async_api import async_playwright

async def test_trace():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()

        # Start tracing
        await context.tracing.start(screenshots=True, snapshots=True)

        page = await context.new_page()
        url = await server.serve()
        await page.goto(url)

        await page.wait_for_timeout(10000)

        # Perform actions
        
        # Stop tracing and save the trace file
        await context.tracing.stop(path="trace.zip")
        print("Trace recorded to trace.zip")

        await browser.close()
        
await test_trace()

Trace recorded to trace.zip


/var/folders/b8/2jd9x_r53sn9l_wjrtwg_0080000gn/T/ipykernel_34050/2617292242.py:23: RuntimeWarning: coroutine 'Browser.close' was never awaited
  browser.close()


In [ ]:
from 

async def trace_logger(
    url: str,
    steps: list[Callable[[Page], Awaitable[None]]],
    snapshot_dir: str = "./website_snapshots",
):
    snapshot_dir: Path = Path(snapshot_dir)
    snapshot_dir.mkdir(parents=True, exist_ok=True)
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # Set headless=True if you don't want a visible browser
        context = await browser.new_context()
        page = await context.new_page()

        # Directory to store snapshots
       

        # Navigate to the initial page
        _ =await page.goto(url)  # Replace with your target website
        await save_snapshot(page, snapshot_dir / "step0")

        # Example actions (trace steps)
        # 1. Click a button (replace with actual selector)
        for i, step in enumerate(steps):  
            try:
                await step(page)
                await save_snapshot(page, snapshot_dir / f"step{i+1}")
            except Exception as e:
                print(f"Step {i+1} failed: {e}")
        # Add more steps as needed
        await browser.close()



In [9]:
server = SnapshotServer(dir_name)
await server.start()


Serving HTTP on port 8080...


In [ ]:
from dataclasses import dataclass, field
from typing import ClassVar
from pathlib import Path

from notte.browser.snapshot import clean_url

@dataclass
class TestSnapshotRouter:
    BASE_DIR: ClassVar[Path] = Path(__file__).parent / ".snapshots"
    ROUTING_TABLE_PATH: ClassVar[Path] = "routing_table.json"
    
    original_url: str
    snapshot_dir: Path
    routing_table: dict[str, Path] = field(init=False)
    
    def __post_init__(self):
        if not self.routing_table_path.exists():
            self.routing_table = {}
            with open(self.routing_table_path, "w") as f:
                json.dump(self.routing_table, f)
        else:
            with open(self.routing_table_path, "r") as f:
                self.routing_table = json.load(f)
    
    @staticmethod
    def from_root_url(url: str) -> "TestSnapshotRouter":
        return TestSnapshotRouter(
            original_url=url,
            snapshot_dir=TestSnapshotRouter.url_base_dir(url)
        )
    
    @property
    def snapshot_url(self) -> str:
        return self.snapshot_dir
    
    @staticmethod
    def url_base_dir(url: str) -> Path:
        clean_url = clean_url(url)
        # transform to camel case
        return self.BASE_DIR / clean_url.replace(".", "_").replace("-", "_")
    
    def add_routing_entry(self, action_id: str, url: str):
        self.routing_table[action_id] = {
            "url": url,
            "snapshot_dir": self.snapshot_dir / action_id
        }
        with open(self.routing_table_path, "w") as f:
            json.dump(self.routing_table, f)
    
    @property
    def routing_table_path(self) -> Path:
        return self.url_base_dir / self.ROUTING_TABLE_PATH
    
    def step(self, action_id: str) -> "TestSnapshotRouter":
        if action_id not in self.routing_table:
            raise ValueError(f"Action ID {action_id} not found in routing table")
        snapshot_info = self.routing_table[action_id]
        return TestSnapshotRouter(
            original_url=snapshot_info["url"],
            snapshot_dir=snapshot_info["snapshot_dir"]
        )
        


from notte.browser.driver import BrowserDriver

class TracingBrowserDriver(BrowserDriver):
    def __init__(self, **browser_kwargs):
        super().__init__(**browser_kwargs)
        self.server = server





In [ ]:


class TestBrowserDriver:
    def __init__(self, server: SnapshotServer):
        self.server = server

    async def run(self):
        with NotteEnv(headless=False) as env:

In [ ]:
with NotteEnv(headless=False) as env:
    await env.goto(server.get_snapshot_url())
